# **Dependencies**#

In [55]:
import pennylane as qml
from pennylane import numpy as np
from torchvision import datasets, transforms
from pennylane.templates import AmplitudeEmbedding
import torchvision
import torch
from sklearn import preprocessing
from sklearn.decomposition import PCA
from tqdm import tqdm

# Set parameters


In [56]:
num_qubits = 5
num_layers = 5
batch_size = 100
epochs = 200
num_classes = 4
dev = qml.device('default.qubit', wires=num_qubits)
keep_labels = [1, 4, 7, 9]


# Setup Data

In [57]:
train_loader = torch.utils.data.DataLoader(datasets.MNIST('../mnist',
                                                          download=True,
                                                          train=True,

                                                          transform=transforms.Compose([
                                                              torchvision.transforms.ToTensor(),
                                                              transforms.Normalize((0.1307,), (0.3081,)),
                                                              transforms.Lambda(torch.flatten),
                                                          ])),
                                           batch_size=10000,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(datasets.MNIST('../mnist',
                                                          download=True,
                                                          train=False,

                                                          transform=transforms.Compose([
                                                              torchvision.transforms.ToTensor(),
                                                              transforms.Normalize((0.1307,), (0.3081,)),
                                                              transforms.Lambda(torch.flatten)
                                                          ])),
                                           batch_size=100,
                                           shuffle=True)

def to_label(label):
  if label == 1:
    return [-1, -1]
  if label == 4:
    return [-1, 1]
  if label == 7:
    return [1, -1]
  if label == 9:
    return [1, 1]
train_data = []
train_labels = []
test_data = []
test_labels = []
for (data, labels) in train_loader:
  for x, y in zip(data, labels):
    if y in keep_labels:
      train_data.append(x.numpy())
      train_labels.append(to_label(y.numpy()))
for (data, labels) in test_loader:
  for x, y in zip(data, labels):
    if y in keep_labels:
      test_data.append(x.numpy())
      test_labels.append(to_label(y.numpy()))
test_data = test_data[:150]
test_labels = test_labels[:150]
pca = PCA(32)
pca.fit(preprocessing.normalize(train_data + test_data))
train_data = pca.transform(preprocessing.normalize(train_data))
test_data = pca.transform(preprocessing.normalize(test_data))

# Define Model

In [58]:
def layer(W, wires, configuration=0):
    for i in range(wires):
        qml.Rot(W[i, 0], W[i, 1], W[i, 2], wires=i)
    if configuration == 0:
        qml.CNOT(wires=[0, 1])
        qml.CNOT(wires=[1, 2])
        qml.CNOT(wires=[2, 3])
        qml.CNOT(wires=[3, 4])
        qml.CNOT(wires=[4, 0])
    elif configuration == 1:
        qml.CNOT(wires=[0, 1])
        qml.CNOT(wires=[1, 2])
        qml.CNOT(wires=[2, 3])
        qml.CNOT(wires=[3, 4])
        qml.CNOT(wires=[4, 0])
        qml.CNOT(wires=[0, 2])
        qml.CNOT(wires=[0, 2])
        qml.CNOT(wires=[0, 1])
    elif configuration == 2:
        qml.CNOT(wires=[0, 1])
        qml.CNOT(wires=[1, 2])
        qml.CNOT(wires=[2, 3])
        qml.CNOT(wires=[3, 4])
        qml.CNOT(wires=[4, 0])
        qml.CNOT(wires=[0, 1])
        qml.CNOT(wires=[3, 1])
        qml.CNOT(wires=[3, 0])
    else:
        qml.CNOT(wires=[0, 1])
        qml.CNOT(wires=[1, 2])
        qml.CNOT(wires=[0, 1])
        qml.CNOT(wires=[2, 3])
        qml.CNOT(wires=[3, 4])
        qml.CNOT(wires=[4, 2])
        qml.CNOT(wires=[4, 0])

In [59]:
def accuracy(labels, predictions):
    bit0 = [np.sign(i[0]) for i in predictions]
    acc = 0
    if num_classes==2:
        for l, p in zip(labels, bit0):
            if abs(l - p) < 1e-5:
                acc = acc + 1
        acc = acc / len(labels)
    elif num_classes==4:
        bit1 = [np.sign(i[1]) for i in predictions]
        for l, p, p1, in zip(labels, bit0, bit1):
            if abs(l[0] - p) < 1e-5 and abs(l[1] - p1) < 1e-5:
                acc = acc + 1
        acc = acc / len(labels)
    else:
        bit1 = [np.sign(i[1]) for i in predictions]
        bit2 = [np.sign(i[2]) for i in predictions]
        for l, p, p1, p2 in zip(labels, bit0, bit1, bit2):
            if abs(l[0] - p) < 1e-5 and abs(l[1] - p1) < 1e-5 and abs(l[2] - p2) < 1e-5:
                acc = acc + 1
        acc = acc / len(labels)
    return acc

def square_loss(labels, predictions):
    loss = 0
    if num_classes == 2:
        for l, p in zip(labels, predictions):
            loss = loss + (l - p[0]) ** 2
        loss = loss / len(labels)
        return loss
    elif num_classes ==4:
        for l, p in zip(labels, predictions):
            loss = loss + ((l[0]-p[0])**2 + (l[1]-p[1])**2)
        loss = loss / len(labels)
        return loss
    else:
        for l, p in zip(labels, predictions):
            loss = loss + ((l[0]-p[0])**2 + (l[1]-p[1])**2+ (l[2]-p[2])**2)
        loss = loss / len(labels)
        return loss
class Quilt:
    def __init__(self, config, weights=None):
      self.config = config
      self.weights = weights if weights is not None else 0.01 * np.random.randn(num_layers, num_qubits, 3)
      self.opt = qml.AdamOptimizer(stepsize=0.05)
      self.losses = []
      self.iter = 0
    def train(self, data, labels):
      for l in tqdm(range(epochs)):
        self.iter = l
        train_indecies= np.random.randint(0, len(data), (batch_size,))
        x_train_batch, y_train_batch = [data[im] for im in train_indecies], [labels[im] for im in train_indecies]
        self.weights = self.opt.step(lambda v: self.cost(v, x_train_batch, y_train_batch), self.weights)
        np.save(f'weights_{self.config}', self.weights)





    @qml.qnode(dev)
    def circuit(self, weights, features):
      image = features.astype('float64')
      AmplitudeEmbedding(image, wires=range(num_qubits), normalize=True, pad_with=0)
      for i in range(num_layers):
        layer(weights[i], num_qubits, self.config)
      return [qml.expval(qml.PauliZ(i)) for i in range(int(np.log2(num_classes)))]


    def cost(self, weights, features, labels):
      predictions = [self.circuit(self, weights, f) for f in features]
      loss = square_loss(labels, predictions)
      acc = accuracy(labels, predictions)
      np.save(f'acc_{self.config}', acc)
      if self.iter %10 == 0:
        print(self.iter, acc)
      return loss
    


# Train Ensemble

In [60]:
ensembles = range(3)
for member in ensembles:
  ens = Quilt(member)
  ens.train(train_data, train_labels)




  0%|          | 0/200 [00:00<?, ?it/s]/Users/danielsilver/Repos/Quilt/venv/lib/python3.9/site-packages/pennylane/math/utils.py:227: UserWarning: Contains tensors of types {'autograd', '__main__'}; dispatch will prioritize TensorFlow, PyTorch, and  Jax over Autograd. Consider replacing Autograd with vanilla NumPy.
  warnings.warn(


0 0.18


  5%|▌         | 10/200 [02:19<44:07, 13.94s/it]

10 0.24


 10%|█         | 20/200 [04:39<42:11, 14.06s/it]

...

 90%|█████████ | 180/200 [6:40:13<1:13:28, 220.43s/it]

180 0.82


 95%|█████████▌| 190/200 [11:44:21<8:09:53, 2939.37s/it]

190 0.8


100%|██████████| 200/200 [16:00:29<00:00, 288.15s/it]   


# Evaluate Ensemble

In [62]:
correct = 0

ensemble_predictions = [0 for _ in range(len(test_data))]
for member in ensembles:
  weights = np.load(f'weights_{member}.npy')
  relative_acc = np.load(f'acc_{member}.npy')
  ens = Quilt(member, weights)
  predictions = [ens.circuit(ens,  weights, f) for f in test_data]
  for p in range(len(predictions)):
    ensemble_predictions[p] += predictions[p] * relative_acc

for i, j in zip(ensemble_predictions, test_labels):
  if np.sign(i[0]) == np.sign(j[0]) and np.sign(i[1]) == np.sign(j[1]) :
    correct +=1
print(f'Accuracy: {round(100*correct/len(test_data))}%')

  
  

Accuracy: 83%
